1. Загрузка библиотек и настройка параметров

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, \
    roc_curve, auc
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import os
import random

In [28]:
# Фиксация всех случайных seed для воспроизводимости
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # если используются многопоточные GPU
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(SEED)  # если используется стандартный модуль random


# Для воспроизводимости DataLoader при использовании многопоточности
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


g = torch.Generator()
g.manual_seed(SEED)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

2. Подготовка и предобработка данных

2.1. Датасеты

В статье используются публичные датасеты, содержащие как нормальный сетевой трафик, так и аномальные/злоумышленные потоки. Например, можно попробовать использовать:

- USTC-TFC (University of Science and Technology of China – Traffic Flow Collection)

- Другие датасеты для сетевого обнаружения аномалий

2.2. Предобработка

Согласно статье, на вход модели подаются первые n пакетов каждого сетевого потока, причем:

- Фильтрация: Выбираются пакеты, принадлежащие одному 5-тuple (IP-адреса, порты, протокол).

- Анонимизация: MAC- и IP-адреса маскируются.

- Трансформация: Каждый пакет приводится к фиксированной длине l (если пакет длиннее – усечение, если короче – дополнение нулями).

- Нормализация: Значения байтов нормализуются (например, делением на 255).

- Конкатенация: n пакетов объединяются в единый входной вектор.

Пример кода для обработки может выглядеть следующим образом (упрощённо):

In [30]:
FEATURE_COLUMNS = [
        "Average Packet Size", "Flow Bytes/s", "Fwd Packet Length Mean", "Max Packet Length",
        "Fwd IAT Min", "Total Length of Fwd Packets", "Flow IAT Mean", "Fwd IAT Std",
        "Fwd Packet Length Max", "Fwd Header Length"
        ]
LABEL_COLUMN = "Label"

In [31]:
class CSVAttackDataset(Dataset):
    def __init__(self, attack_dir, interval, data_type='normal'):
        """
        attack_dir: путь к папке с CSV-файлами для конкретной атаки (например, 'CIC-IDS-2017/attacks/botnet-ares/csvs')
        interval: длина пакетного интервала (например, '1000')
        data_type: 'attack' или 'normal'
        """
        self.data = self._load_data(attack_dir, interval, data_type)

    def _load_data(self, attack_dir, interval, data_type):
        data_frames = []
        data_path = os.path.join(attack_dir, data_type)
        if not os.path.exists(data_path):
            return pd.DataFrame()

        # Фильтрация файлов по интервалу и типу данных
        for file_name in os.listdir(data_path):
            if f"_{interval}_" in file_name and file_name.endswith('.csv'):
                file_path = os.path.join(data_path, file_name)
                df = pd.read_csv(file_path)
                data_frames.append(df)

        if data_frames:
            return pd.concat(data_frames, ignore_index=True)
        else:
            return pd.DataFrame()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        features = sample[FEATURE_COLUMNS].values.astype(np.float32) / 255.0
        tensor = torch.tensor(features, dtype=torch.float32)
        return tensor.unsqueeze(0)

In [32]:
class CombinedTestDataset(Dataset):
    def __init__(self, attack_dir, interval):
        self.data = self._load_data(attack_dir, interval)

    def _load_data(self, attack_dir, interval):
        data_frames = []
        for data_type in ['attack', 'normal']:
            data_path = os.path.join(attack_dir, data_type)
            if not os.path.exists(data_path):
                continue

            for file_name in os.listdir(data_path):
                if f"_{interval}_" in file_name and file_name.endswith('.csv'):
                    file_path = os.path.join(data_path, file_name)
                    df = pd.read_csv(file_path)
                    data_frames.append(df)

        if data_frames:
            return pd.concat(data_frames, ignore_index=True)
        else:
            return pd.DataFrame()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        features = sample[FEATURE_COLUMNS].values.astype(np.float32) / 255.0
        label = sample[LABEL_COLUMN]
        return torch.tensor(features).unsqueeze(0), torch.tensor(label, dtype=torch.float32)

3. Модель и архитектура

ARCADE состоит из двух компонентов:

3.1. Автоэнкодер (AE)

Используется компактная свёрточная архитектура (inspired by DCGAN) для кодирования и декодирования входных сетевых потоков.

Основная идея:

- Encoder: Преобразует входной сигнал в компактное латентное представление.

- Decoder: Восстанавливает исходный сигнал из латентного представления.

3.2. Критик (Critic)

Отдельная нейросеть, выполняющая роль дискриминатора (как в WGAN-GP), оценивающая, насколько реконструкция (выход AE) соответствует нормальному распределению трафика. При обучении используется штраф за градиент (gradient penalty) для соблюдения условий липшицевости.

In [33]:
class Encoder(nn.Module):
    def __init__(self, latent_dim=50):
        super(Encoder, self).__init__()

        self.model = nn.Sequential(
                # Вход: (batch, 1, 10)
                nn.Conv1d(1, 16, kernel_size=4, stride=2, padding=1),  # Выход: (16, 5)
                nn.BatchNorm1d(16),
                nn.LeakyReLU(),

                nn.Conv1d(16, 32, kernel_size=4, stride=2, padding=1),  # Выход: (32, 2)
                nn.BatchNorm1d(32),
                nn.LeakyReLU(),

                nn.Conv1d(32, 64, kernel_size=4, stride=2, padding=2),  # Изменён padding=2 → Выход: (64, 2)
                nn.BatchNorm1d(64),
                nn.LeakyReLU(),

                nn.Flatten(),
                nn.Linear(64 * 2, latent_dim, bias=False)  # Теперь 64*2=128 входов
                )

    def forward(self, x):
        return self.model(x)

In [34]:
class Decoder(nn.Module):
    def __init__(self, latent_dim=50):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
                nn.Linear(latent_dim, 64 * 2, bias=False),  # Вход:50 → Выход:128
                nn.Unflatten(1, (64, 2)),  # Восстановление (64, 2)

                nn.BatchNorm1d(64),
                nn.ReLU(),

                # Первый ConvTranspose: (64,2) → (32,5)
                nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1, output_padding=1),
                nn.BatchNorm1d(32),
                nn.ReLU(),

                # Второй ConvTranspose: (32,5) → (16,10)
                nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1),
                nn.BatchNorm1d(16),
                nn.ReLU(),

                # Финал: (16,10) → (1,10)
                nn.Conv1d(16, 1, kernel_size=1, stride=1),
                nn.Sigmoid()
                )

    def forward(self, x):
        return self.model(x)

In [35]:
class Autoencoder(nn.Module):
    def __init__(self, latent_dim=50):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def forward(self, x):
        latent = self.encoder(x)
        reconstruction = self.decoder(latent)
        return reconstruction

In [36]:
class Critic(nn.Module):
    def __init__(self, latent_dim=50):
        super(Critic, self).__init__()

        self.model = nn.Sequential(
                nn.Conv1d(1, 16, kernel_size=4, stride=2, padding=1),  # (16,5)
                nn.LayerNorm([16, 5]),
                nn.LeakyReLU(),

                nn.Conv1d(16, 32, kernel_size=4, stride=2, padding=1),  # (32,2)
                nn.LayerNorm([32, 2]),
                nn.LeakyReLU(),

                nn.Conv1d(32, 64, kernel_size=4, stride=2, padding=2),  # (64,2) с padding=2
                nn.LayerNorm([64, 2]),
                nn.LeakyReLU(),

                nn.Flatten(),
                nn.Linear(64 * 2, latent_dim, bias=False),  # 64*2=128 входов
                nn.LayerNorm(latent_dim),
                nn.LeakyReLU(),

                nn.Linear(latent_dim, 1)
                )

    def forward(self, x):
        return self.model(x)

4. Функции потерь и штраф за градиент

4.1. Функция реконструкции

Чаще всего используется MSE (Mean Squared Error) или L1 loss:

4.2. Adversarial loss (WGAN-GP)

Критик обучается по следующему принципу:

- Максимизировать оценку для реальных нормальных потоков.

- Минимизировать оценку для реконструкций, полученных автоэнкодером.

При этом применяется штраф за градиент, как описано в статье (см. уравнение (4)).

Пример функции для расчёта gradient penalty:

In [37]:
def compute_gradient_penalty(critic, real_data, fake_data, device, lambda_gp=10):
    batch_size = real_data.size(0)
    alpha = torch.rand(batch_size, 1, 1, device=device)
    interpolates = alpha * real_data + ((1 - alpha) * fake_data)
    interpolates.requires_grad_(True)
    critic_interpolates = critic(interpolates)
    gradients = grad(
            outputs=critic_interpolates,
            inputs=interpolates,
            grad_outputs=torch.ones_like(critic_interpolates, device=device),
            create_graph=True,
            retain_graph=True,
            only_inputs=True
            )[0]
    gradients = gradients.view(batch_size, -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lambda_gp
    return gradient_penalty

5. Процедура обучения

5.1. Стратегия обучения

Архитектура обучается по схеме adversarial training:

- Шаг 1. Обучение критика:
    - Используйте реальные нормальные данные и реконструкции автоэнкодера.

    - Обновляйте веса критика, максимизируя разницу между оценками (реальные – реконструированные), добавляя штраф за градиент.

- Шаг 2. Обучение автоэнкодера:

    - Минимизируйте комбинированную функцию потерь, состоящую из ошибки реконструкции и adversarial loss (то есть, ошибка между оценкой критика для реконструкций и целевым значением).

5.2. Пример цикла обучения

Ниже приведён псевдокод, демонстрирующий общий цикл обучения:

In [38]:
def evaluate_validation(autoencoder, val_loader, device, reconstruction_criterion):
    autoencoder.eval()
    val_loss = 0.0
    count = 0
    with torch.no_grad():
        for data in val_loader:
            data = data.to(device)
            reconstruction = autoencoder(data)
            loss = reconstruction_criterion(reconstruction, data)
            val_loss += loss.item() * data.size(0)
            count += data.size(0)
    autoencoder.train()
    return val_loss / count

In [39]:
def train_arcade(autoencoder, critic, train_loader, val_loader, device,
                 searching_epochs=100, fine_tuning_epochs=50,
                 lr_searching=1e-4, lr_finetuning=1e-5,
                 lambda_gp=10, lambda_adv=0.01,
                 critic_iters=5,
                 balance_threshold=0.001, balance_patience=5,
                 early_stopping_patience=10
                 ):
    total_epochs = searching_epochs + fine_tuning_epochs

    # Инициализация оптимизаторов с β1=0, β2=0.9
    optimizer_AE = optim.Adam(autoencoder.parameters(), lr=lr_searching, betas=(0, 0.9))
    optimizer_Critic = optim.Adam(critic.parameters(), lr=lr_searching, betas=(0, 0.9))

    reconstruction_criterion = nn.MSELoss()

    best_val_loss = float('inf')
    epochs_without_improvement = 0
    balance_counter = 0  # Количество последовательных эпох, когда разница лоссов меньше порога

    epoch = 0
    while epoch < total_epochs:
        total_loss_ae = 0.0
        total_loss_critic = 0.0
        num_batches = 0

        # Если перешли в фазу fine-tuning, обновляем lr оптимизаторов
        if epoch == searching_epochs:
            for param_group in optimizer_AE.param_groups:
                param_group['lr'] = lr_finetuning
            for param_group in optimizer_Critic.param_groups:
                param_group['lr'] = lr_finetuning

        # batch_pbar = tqdm(enumerate(train_loader), total=len(train_loader),
        #                   desc=f"Epoch {epoch + 1}/{total_epochs}", unit="batch", leave=False
        #                   )

        for i, real_data in enumerate(train_loader):
            real_data = real_data.to(device)

            # Обновление критика (несколько итераций)
            critic_loss_epoch = 0.0
            for _ in range(critic_iters):
                optimizer_Critic.zero_grad()
                reconstruction = autoencoder(real_data)
                critic_real = critic(real_data)
                critic_fake = critic(reconstruction.detach())
                gp = compute_gradient_penalty(critic, real_data, reconstruction.detach(), device, lambda_gp)
                loss_critic = -(torch.mean(critic_real) - torch.mean(critic_fake)) + gp
                loss_critic.backward()
                optimizer_Critic.step()
                critic_loss_epoch += loss_critic.item()
            avg_critic_loss = critic_loss_epoch / critic_iters

            # Обновление автоэнкодера
            optimizer_AE.zero_grad()
            reconstruction = autoencoder(real_data)
            loss_reconstruction = reconstruction_criterion(reconstruction, real_data)
            loss_adv = -torch.mean(critic(reconstruction))
            loss_AE = loss_reconstruction + lambda_adv * loss_adv
            loss_AE.backward()
            optimizer_AE.step()

            total_loss_ae += loss_AE.item()
            total_loss_critic += avg_critic_loss
            num_batches += 1

            # batch_pbar.set_postfix({
            #         "MSE Loss"   : f"{loss_reconstruction:.4f}",
            #         "AE Loss"    : f"{loss_AE.item():.4f}",
            #         "Critic Loss": f"{avg_critic_loss:.4f}"
            #         }
            #         )
        # batch_pbar.close()

        avg_ae_loss = total_loss_ae / num_batches
        avg_critic_loss = total_loss_critic / num_batches

        # Проверка условия балансировки лоссов
        if abs(avg_ae_loss - avg_critic_loss) < balance_threshold:
            balance_counter += 1
        else:
            balance_counter = 0

        # Вычисляем ошибку на валидационном наборе
        val_loss = evaluate_validation(autoencoder, val_loader, device, reconstruction_criterion)
        print(
                f"- Epoch {epoch + 1}: Avg AE Loss = {avg_ae_loss:.4f}, Avg Critic Loss = {avg_critic_loss:.4f}, Val Loss = {val_loss:.4f}"
                )

        # Раннее прекращение по валидации
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        # Если баланс достигнут в течение balance_patience эпох или нет улучшения валидации
        if balance_counter >= balance_patience:
            print("Баланс между лоссами достигнут, остановка обучения.")
            break
        if epochs_without_improvement >= early_stopping_patience:
            print("Нет улучшения на валидации, early stopping.")
            break

        epoch += 1
    print("Обучение завершено.")

In [40]:
def evaluate_metrics(autoencoder, dataloader, device):
    all_scores = []
    all_labels = []
    autoencoder.eval()

    # Сбор ошибок реконструкции и меток для всех примеров
    with torch.no_grad():
        for data, labels in dataloader:
            data = data.to(device)
            reconstruction = autoencoder(data)
            # Вычисление ошибки реконструкции (например, среднеквадратичная ошибка по каждому примеру)
            scores = torch.mean((data - reconstruction) ** 2, dim=[1, 2]).cpu().numpy()
            all_scores.extend(scores)
            all_labels.extend(labels.cpu().numpy())

    all_scores = np.array(all_scores)
    all_labels = np.array(all_labels)

    # Вычисление AUROC без пороговой бинаризации
    auroc = roc_auc_score(all_labels, all_scores)

    # Подбор порога на основе максимизации F1-метрики
    thresholds = np.linspace(np.min(all_scores), np.max(all_scores), num=100)
    f1_scores = []
    for thr in thresholds:
        preds = (all_scores > thr).astype(int)
        f1 = f1_score(all_labels, preds)
        f1_scores.append(f1)
    best_threshold = thresholds[np.argmax(f1_scores)]

    # Применение оптимального порога для вычисления остальных метрик
    predictions = (all_scores > best_threshold).astype(int)
    accuracy = accuracy_score(all_labels, predictions)
    precision = precision_score(all_labels, predictions)
    recall = recall_score(all_labels, predictions)
    f1 = f1_score(all_labels, predictions)

    # Вычисление матрицы ошибок
    tn, fp, fn, tp = confusion_matrix(all_labels, predictions).ravel()

    print(f"AUROC: {auroc:.4f}")
    print(f"Лучший threshold: {best_threshold:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1: {f1:.4f}")
    print(f"Матрица ошибок:")
    print(f"TN: {tn:.4f}")
    print(f"FP: {fp:.4f}")
    print(f"FN: {fn:.4f}")
    print(f"TP: {tp:.4f}")

    # ------------------ Построение графиков ------------------

    # 1. Гистограмма распределения ошибок реконструкции
    plt.figure(figsize=(12, 5))
    plt.hist(all_scores[all_labels == 0], bins=50, alpha=0.6, label='Нормальные')
    plt.hist(all_scores[all_labels == 1], bins=50, alpha=0.6, label='Аномалии')
    plt.axvline(best_threshold, color='red', linestyle='--', label=f'Лучший threshold: {best_threshold:.4f}')
    plt.xlabel('Ошибка реконструкции')
    plt.ylabel('Количество примеров')
    plt.title('Распределение ошибок реконструкции')
    plt.legend()
    plt.show()

    # 2. ROC-кривая
    fpr, tpr, _ = roc_curve(all_labels, all_scores)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, label=f'ROC кривая (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC кривая')
    plt.legend(loc='lower right')
    plt.show()

    # 3. Зависимость F1-score от порога
    plt.figure(figsize=(8, 5))
    plt.plot(thresholds, f1_scores, marker='o')
    plt.xlabel('Threshold')
    plt.ylabel('F1-score')
    plt.title('Зависимость F1-score от порога')
    plt.axvline(best_threshold, color='red', linestyle='--', label=f'Лучший threshold: {best_threshold:.4f}')
    plt.legend()
    plt.show()

    return auroc, predictions, accuracy, precision, recall, f1, tn, fp, fn, tp, roc_auc

In [41]:
results = []

In [42]:
def train_attack_category(attack_category, base_dir='../CIC-IDS-2017/attacks'):
    attack_dir = os.path.join(base_dir, attack_category, 'csvs')
    intervals = set()

    for data_type in ['attack', 'normal']:
        data_path = os.path.join(attack_dir, data_type)
        if os.path.exists(data_path):
            for file_name in os.listdir(data_path):
                parts = file_name.split('_')
                if len(parts) > 2:
                    interval = parts[-3]
                    intervals.add(interval)

    for interval in sorted(intervals):
        if interval != '10000':
            for latent_dim in [10]:
                print(
                        f"\n< ---------- {attack_category} | Interval: {interval} | Latent Dim: {latent_dim} ----------- >"
                        )

                full_dataset = CSVAttackDataset(attack_dir, interval, data_type='normal')
                train_size = int(0.9 * len(full_dataset))
                val_size = len(full_dataset) - train_size
                train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

                test_dataset = CombinedTestDataset(attack_dir, interval)

                batch_size = 32

                train_dataloader = DataLoader(
                        train_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        worker_init_fn=seed_worker,  # <- для воркеров
                        generator=g,  # <- для перемешивания
                        num_workers=4
                        )

                val_dataloader = DataLoader(
                        val_dataset,
                        batch_size=batch_size,
                        shuffle=False,
                        worker_init_fn=seed_worker,  # важно, если есть аугментации
                        generator=g  # не влияет при shuffle=False, но для единообразия
                        )

                test_dataloader = DataLoader(
                        test_dataset,
                        batch_size=batch_size,
                        shuffle=False,
                        worker_init_fn=seed_worker,  # если есть преобразования данных
                        generator=g  # можно опустить, но лучше оставить
                        )

                autoencoder = Autoencoder().to(device)
                critic = Critic().to(device)

                train_arcade(autoencoder, critic, train_dataloader, val_dataloader, device)

                auroc, predictions, accuracy, precision, recall, f1, tn, fp, fn, tp, roc_auc = evaluate_metrics(
                        autoencoder, test_dataloader, device
                        )

                results.append({
                        'LatentDim'      : latent_dim,
                        'Attack Category': attack_category,
                        'Interval'       : interval,
                        'AUROC'          : auroc,
                        'Accuracy'       : accuracy,
                        'Precision'      : precision,
                        'Recall'         : recall,
                        'F1 Score'       : f1,
                        'True Negative'  : tn,
                        'False Positive' : fp,
                        'False Negative' : fn,
                        'True Positive'  : tp,
                        'ROC AUC'        : roc_auc
                        }
                        )

    results_df = pd.DataFrame(results)
    results_df.to_csv(f'{attack_category}_results-2.csv', index=False)
    print(f"Результаты сохранены в {attack_category}_results-2.csv")
    return results_df

In [43]:
train_attack_category('botnet-ares')


< ---------- botnet-ares | Interval: 10 | Latent Dim: 10 ----------- >
- Epoch 1: Avg AE Loss = 52108022.1276, Avg Critic Loss = -1.8788, Val Loss = 51929859.6900
- Epoch 2: Avg AE Loss = 52107379.1166, Avg Critic Loss = -13.3902, Val Loss = 51929842.9415
- Epoch 3: Avg AE Loss = 52115209.1848, Avg Critic Loss = -27.6407, Val Loss = 51929838.7403
- Epoch 4: Avg AE Loss = 52109551.7969, Avg Critic Loss = -44.8511, Val Loss = 51929835.2580
- Epoch 5: Avg AE Loss = 52100015.1202, Avg Critic Loss = -65.5635, Val Loss = 51929834.8359
- Epoch 6: Avg AE Loss = 52102759.6829, Avg Critic Loss = -89.1003, Val Loss = 51929834.4666
- Epoch 7: Avg AE Loss = 52117581.6041, Avg Critic Loss = -114.7884, Val Loss = 51929833.9390
- Epoch 8: Avg AE Loss = 52113769.6804, Avg Critic Loss = -143.3679, Val Loss = 51929833.8862
- Epoch 9: Avg AE Loss = 52102561.8127, Avg Critic Loss = -175.3836, Val Loss = 51929834.0973


KeyboardInterrupt: 

In [18]:
train_attack_category('brute-ftp')

In [19]:
train_attack_category('brute-http')


< ---------- brute-http | Interval: 10 | Latent Dim: 10 ----------- >
Нет улучшения на валидации, early stopping.
Обучение завершено.
AUROC: 0.2364
Лучший threshold: 0.2858
Accuracy: 0.7832
Precision: 0.7831
Recall: 1.0000
F1: 0.8784
Матрица ошибок:
TN: 1.0000
FP: 1084.0000
FN: 0.0000
TP: 3914.0000

< ---------- brute-http | Interval: 10 | Latent Dim: 50 ----------- >
Нет улучшения на валидации, early stopping.
Обучение завершено.
AUROC: 0.2364
Лучший threshold: 0.3536
Accuracy: 0.7832
Precision: 0.7831
Recall: 1.0000
F1: 0.8784
Матрица ошибок:
TN: 1.0000
FP: 1084.0000
FN: 0.0000
TP: 3914.0000

< ---------- brute-http | Interval: 10 | Latent Dim: 100 ----------- >
Нет улучшения на валидации, early stopping.
Обучение завершено.
AUROC: 0.2365
Лучший threshold: 0.3548
Accuracy: 0.7832
Precision: 0.7831
Recall: 1.0000
F1: 0.8784
Матрица ошибок:
TN: 1.0000
FP: 1084.0000
FN: 0.0000
TP: 3914.0000

< ---------- brute-http | Interval: 100 | Latent Dim: 10 ----------- >
Нет улучшения на валидац

,LatentDim,Attack Category,Interval,AUROC,Accuracy,Precision,Recall,F1 Score,True Negative,False Positive,False Negative,True Positive,ROC AUC
0,10,brute-http,10,0.236409,0.783157,0.783113,1.000000,0.878366,1,1084,0,3914,0.236409
1,50,brute-http,10,0.236361,0.783157,0.783113,1.000000,0.878366,1,1084,0,3914,0.236361
2,100,brute-http,10,0.236501,0.783157,0.783113,1.000000,0.878366,1,1084,0,3914,0.236501
3,10,brute-http,100,0.732050,0.785571,0.785141,1.000000,0.879640,1,107,0,391,0.732050
4,50,brute-http,100,0.731931,0.785571,0.785141,1.000000,0.879640,1,107,0,391,0.731931
5,100,brute-http,100,0.731979,0.785571,0.785141,1.000000,0.879640,1,107,0,391,0.731979
6,10,brute-http,1000,0.966667,0.959184,0.951220,1.000000,0.975000,8,2,0,39,0.966667
7,50,brute-http,1000,0.966667,0.959184,0.951220,1.000000,0.975000,8,2,0,39,0.966667
8,100,brute-http,1000,0.966667,0.959184,0.951220,1.000000,0.975000,8,2,0,39,0.966667
9,10,brute-http,250,0.830501,0.849246,0.838710,1.000000,0.912281,13,30,0,156,0.830501


In [20]:
train_attack_category('brute-ssh')

In [21]:
train_attack_category('inf-dropbox')

In [22]:
train_attack_category('inf-usb')

In [23]:
train_attack_category('sql-inj')

In [24]:
train_attack_category('xss')

In [25]:
# Создаем DataFrame и сохраняем в CSV
base_dir = 'Statistics-n_packets=5_l=100'
results_df = pd.DataFrame(results)
results_df.to_csv(f'{base_dir}_attack_metrics_report.csv', index=False)
print(f"\nMetrics saved to {base_dir}_attack_metrics_report.csv")

In [26]:
results_df